In [1]:
using Pkg
# Pkg.activate("/Users/ryansenne/Documents/Github/SSM/")
Pkg.activate("/home/ryansenne/PycharmProjects/ssm_julia/")

  Activating project at `~/PycharmProjects/ssm_julia`


In [2]:
# include("/Users/ryansenne/Documents/Github/SSM//src/SSM.jl")
include("/home/ryansenne/PycharmProjects/ssm_julia/src/SSM.jl")
using Random
using .SSM
using Distributions

### Gaussian GLM Example

In [3]:
# Fit a Gaussian GLM
# Generate some synthetic data
X = rand(10000, 3);  # 100 samples, 3 features;
# Add a column of ones to represent the intercept term
X_concat = hcat(ones(10000), X)
β = [2, 3, -4, 5]
y = X_concat * β + rand(Normal(0, 1), 10000)

# define a GLM
glm = GaussianRegression(X, y)
fit!(glm, LSELoss(), 1000)

4-element Vector{Float64}:
  2.0187908479131846
  2.992330199639041
 -4.047993305173819
  4.991399105504719

### Poisson GLM Example

In [4]:
# Fit a Poisson GLM
# Generate some synthetic data
X = rand(10000, 3);  # 100 samples, 3 features;
# Add a column of ones to represent the intercept term
X_concat = hcat(ones(10000), X)
β = [2, 3, -4, 5]

# Generate some synthetic data
y = Vector{Float64}(rand.(Poisson.(exp.(X_concat * β))))

# define a GLM
glm = PoissonRegression(X, y)
fit!(glm, PoissonLoss(), 1000)


4-element Vector{Float64}:
  1.9993518021288263
  3.0001202482828835
 -4.002665054605984
  5.001981947833053

### Binomial GLM Example (i.e. logistic regression)

In [5]:
# sigmoid function
function sigmoid(x)
    return 1 / (1 + exp(-x))
end

# Fit a Binomial GLM
# Generate some synthetic data
X = rand(10000, 3); # 1000 samples, 3 features;
# Add a column of ones to represent the intercept term
X_concat = hcat(ones(10000), X)
β = [2, 3, -4, 5]

# Generate some synthetic data
y = Vector{Float64}(rand.(Binomial.(1, sigmoid.(X_concat * β))))

# define a GLM
glm = BinomialRegression(X, y)
fit!(glm, CrossEntropyLoss(), 1000)

4-element Vector{Float64}:
  2.0830865982345195
  3.045177366762648
 -4.010889466299791
  4.74474818785333

In [6]:
# try out markov glm

# Generate synthetic data from two distinct regimes
X = rand(10000, 3); # 1000 samples, 3 features;
# Add a column of ones to represent the intercept term
X_concat = hcat(ones(10000), X)
β₁ = [-2, -3, -4, 5]
β₂ = [6, 7, -8, 20]

# Generate some synthetic data
y₁ = X_concat * β₁ + rand(Normal(0, 0.1), 10000)
y₂ = X_concat * β₂ + rand(Normal(0, 1), 10000)

# concatenate X1 and X2
x = vcat(X, X)

# concatenate y1 and y2
y = vcat(y₁, y₂)

# define a Gaussian HMMGLM
glm = GaussianMarkovRegression(y, x, 2)

MarkovRegressionEM(glm)

Iteration: 1, Log-likelihood: -70547.17003100985
Iteration: 2, Log-likelihood: -69756.22866662563
Iteration: 3, Log-likelihood: -68910.67317956932


Iteration: 4, Log-likelihood: -68253.34481469865
Iteration: 5, Log-likelihood: -67963.62801957631
Iteration: 6, Log-likelihood: -67885.07988175843


Iteration: 7, Log-likelihood: -67865.32005133774
Iteration: 8, Log-likelihood: -67859.74635471913
Iteration: 9, Log-likelihood: -67858.03774742776


Iteration: 10, Log-likelihood: -67857.49074083574
Iteration: 11, Log-likelihood: -67857.31154219204
Iteration: 12, Log-likelihood: -67857.25208504437


Iteration: 13, Log-likelihood: -67857.23221454941
Iteration: 14, Log-likelihood: -67857.22554623813
Iteration: 15, Log-likelihood: -67857.22330302895


Iteration: 16, Log-likelihood: -67857.22254734478
Convergence reached at iteration 16


93.1685426582240893.1685417273194

In [ ]:
glm.B[1].regression_model.β

In [ ]:
using Plots
plot(y)